In [1]:
%load_ext tensorboard

import os
import csv
import numpy as np
import tensorflow as tf
import tensorboard as tb

from glob import glob
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

import torch
import torchaudio
from speechbrain.pretrained import EncoderClassifier
from torch.utils.tensorboard import SummaryWriter

C:\Users\Admin\anaconda3\envs\tensorboard\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.


In [2]:
RUN_NAME="runs/spkr_ident_ref"

In [ ]:
writer = SummaryWriter(RUN_NAME)
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [ ]:
def get_embeddings(filepath):
    classifier = EncoderClassifier.from_hparams(
            source=r"../../../models/sb_ecapa_pretrained",
            savedir=r"../../../models/sb_ecapa_pretrained",
        )
    signal, _ = torchaudio.load(filepath)
    embeddings = classifier.encode_batch(signal)
    embeddings = torch.flatten(embeddings)
    return embeddings

In [ ]:
def iterate_all_files(dir):
    embeddings = []
    spkrs = []
    audio_files = glob(dir)
    for file in tqdm(audio_files):
        file_dir = os.path.dirname(file)
        spkr_name = file_dir.split(os.path.sep)[-1]  # Spkr Name (str)
        embedding = get_embeddings(file) # Tensors 
        
        embeddings.append(embedding)
        spkrs.append(spkr_name)
    return spkrs, embeddings

In [ ]:
audio_files_dir = r"../../data/ref_audio/**/*.wav"
spkrs, embeddings = iterate_all_files(audio_files_dir)

In [ ]:
embeddings_reshape = torch.stack(embeddings, 0)
embeddings_reshape.shape

In [ ]:
len(spkrs)

In [ ]:
writer.add_embedding(embeddings_reshape, spkrs)
writer.close()

In [4]:
%tensorboard --logdir $RUN_NAME --port=6008

In [ ]:
images_pil = []
audio_embeddings = []
labels = []

img_path = "../../data/ref_images/anwar.jpg"
img_tf = get_img(img_path)
# Save both tf image for prediction and PIL image for sprite
# img_pil = Image.fromarray(np.uint8(img_tf.numpy())*255).resize((100, 100))
img_pil = img_tf.numpy()
# write to tensorboard
writer.add_image('test', img_pil)

# audio_embedding = get_embeddings("../../data/ref_audio/YB DATO SERI ANWAR BIN IBRAHIM/NQZap4wJ3rU.wav")
# audio_embeddings.append(audio_embedding.numpy()[0])
# images_pil.append(img_pil)
# # Assuming your output data is directly the label
# label = "Anwar" 
# labels.append(label)

In [ ]:
def get_img(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize_with_pad(img, 100, 100)
    return img

In [ ]:
# helper function
def select_n_random(data, labels, n=100):
    '''
    Selects n random datapoints and their corresponding labels from a dataset
    '''
    assert len(data) == len(labels)

    perm = torch.randperm(len(data))
    return data[perm][:n], labels[perm][:n]

# select random images and their target indices
images, labels = select_n_random(trainset.data, trainset.targets)

# get the class labels for each image
class_labels = [classes[lab] for lab in labels]

# log embeddings
features = images.view(-1, 28 * 28)
writer.add_embedding(features,
                    metadata=class_labels,
                    label_img=images.unsqueeze(1))
writer.close()